In [ ]:
import os
import leafmap
import torch
from samgeo import SamGeo, tms_to_geotiff

In [ ]:
m = leafmap.Map(center=[60.619237, 23.228611], zoom=16)
m.add_basemap('SATELLITE')

In [ ]:
m

In [ ]:
if m.user_roi_bounds() is not None:
    bbox = m.user_roi_bounds()
else:
    bbox = [23.226, 60.6184, 23.231, 60.6202]

In [ ]:
image = '../data/satellite.tif'

In [ ]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source='Satellite')

In [ ]:
m.add_raster(image, layer_name='Image')

In [ ]:
checkpoint = os.path.join('../data', 'sam_vit_h_4b8939.pth')

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sam = SamGeo(
    checkpoint=checkpoint,
    model_type='vit_h',
    device=device,
    erosion_kernel=(3, 3),
    mask_multiplier=255,
    sam_kwargs=None,
)

In [ ]:
mask = '../data/segment.tiff'
sam.generate(image, mask)

In [ ]:
vector = '../data/segment.gpkg'
sam.tiff_to_gpkg(mask, vector, simplify_tolerance=None)

In [ ]:
shapefile = '../data/segment.shp'
sam.tiff_to_vector(mask, shapefile)

In [ ]:
style = {
    'color': '#3388ff',
    'weight': 2,
    'fillColor': '#7c4185',
    'fillOpacity': 0.5,
}
m.add_vector(vector, layer_name='Vector', style=style)
m